# Super Learner Class
# Luke Killoran

## Setup:

### Imports

In [ ]:
import os
os.chdir("C:/Users/Luke/Documents/UCD/AML/Labs/Lab 1")
import random
from random import randrange
import pandas as pd
import numpy as np
from sklearn.tree import export_graphviz
from sklearn import metrics
from sklearn import tree
from sklearn import svm
from sklearn import ensemble
from sklearn import linear_model
from sklearn import neighbors
from sklearn import neural_network
from sklearn import naive_bayes
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from itertools import chain, combinations
from sklearn.base import BaseEstimator, ClassifierMixin
from IPython.display import display, HTML, Image
import numpy.random
from collections import Counter

#### Function that reads in data:
Converts csv to dataframe and samples from according to sample rate.
Also normalises the X values by dividing by max pixel value (255) - note 0 corresponds to pure white and 255 to pure black

In [ ]:
def Read_In_File(filename, sample_rate):
    """
    Reads in file by path and samples from it
    Divides all X values (after col 1) by 255 to normalise
    """
    dataset = pd.read_csv(filename)
    dataset = pd.DataFrame(data=dataset)
    dataset = dataset.sample(frac=sample_rate)
    dataset[dataset.columns[1:]] = dataset[dataset.columns[1:]] /255
    return(dataset)

#### Function that takes data and no. of folds and produces dict with those folds

In [ ]:
def Divide_Into_Folds(data,num_folds,random_seed=5):
    """
    Takes data and number of folds and returns dict with each entry a fold 
    """
    for n in range(1,num_folds+1):
        locals()['fold'+str(n)] = []
    library_of_folds = dict()
    obs_each = data.shape[0]//num_folds # at least how many obs each fold
    extra = data.shape[0]%num_folds # spread these extra ones across folds
    random.seed(random_seed)
    randomlist = random.sample(range(len(data)),len(data))
    counter = 0
    for n in range(1,extra+1):
        locals()['fold'+str(n)] += [data.iloc[row] for row in randomlist[counter:counter+obs_each+1]]
        counter = counter+obs_each+1
    for n in range(extra+1,num_folds+1):
        locals()['fold'+str(n)] += [data.iloc[row] for row in randomlist[counter:counter+obs_each]]
        counter = counter+obs_each
    for n in range(1,num_folds+1):
        locals()['fold'+str(n)] = pd.DataFrame(data = locals()['fold'+str(n)])
        library_of_folds[n] = locals()['fold'+str(n)]
    return(library_of_folds)

#### Functions that returns a model based on some text entry (used for brevity)

In [ ]:
def BaseModelName(name,criterion="entropy",min_samples_split = 200, \
                  min_samples_leaf = 50, max_depth=12, n_neighbors=20):
    """
    Returns model function for string entry specifying model
    Can also specify hyperparameters too
    """
    if name == 'dt':
        return(tree.DecisionTreeClassifier(criterion=criterion, \
                                           min_samples_split = min_samples_split, \
                                           min_samples_leaf = min_samples_leaf, \
                                           max_depth=max_depth))
    if name == 'lr':
        return(linear_model.LogisticRegression(multi_class='ovr', C=1, solver='liblinear', max_iter=1000))
    elif name == 'knn':
        return(neighbors.KNeighborsClassifier(n_neighbors=n_neighbors))
    elif name == 'mlp':
        return(neural_network.MLPClassifier(hidden_layer_sizes=(300, 100)))
    elif name == 'svm':
        return(svm.LinearSVC())
    elif name == 'nb':
        return(naive_bayes.GaussianNB())
    else:
        return("Error: Input Not A Correct Model Name")

### Load & Partition Data

Note that I read in a different dataset than the normal MNIST train dataset. I read in the first 7500 rows of that dataset, simply because I would get a Memory Error if I attempted to read in the whole dataset.

In [ ]:
all_data = Read_In_File('fashion-mnist_train-cut.csv', 1)
all_data = all_data.reset_index(drop=True)
num_classes = 10
classes = {0: "T-shirt/top", 1:"Trouser", 2: "Pullover", 3:"Dress", 4:"Coat", 5:"Sandal", 6:"Shirt", 7:"Sneaker", 8:"Bag", 9:"Ankle boot"}

Now, expectedly the distribution of labels is not exactly distributed equally among the dataset so I ran through each row and deleted it if the count of the label of that row exceeded the minimum label count. 

In [ ]:
for row in range(len(all_data)):
    counter = Counter(all_data["label"])
    label = all_data["label"][row]
    if counter[label] > min(counter.values()):
        all_data = all_data.drop(row)
        row -= 1

I then split the resulting dataset into X and Y datasets, and divided them further into training and test sets, both stratified by the label count in each.

In [ ]:
X = all_data[all_data.columns[1:]]
Y = all_data[all_data.columns[0]]
labels = list(Counter(all_data["label"]).keys())
X_train, X_test, Y_train, Y_test \
= train_test_split(X, Y, random_state=89, train_size = 0.8, stratify = Y)

### Function that returns a bootstrapped sample
So sample with replacement, and size of sample determined by inputted fraction of size of dataset

In [ ]:
def Bootstrap(data, size_as_fraction):
    """
    Takes in data (assumes data frame) and returns sample of data 
    (with size of sample specified by size_as_fraction and sample with replacement)
    """
    n = len(data)
    index = numpy.random.randint(0, n, n)
    sample = []
    sample += [data.iloc[row] for row in index]
    sample = pd.DataFrame(sample)
    sample = sample.sample(frac=size_as_fraction)
    return (sample)

### Super Learner Class
This class defined by the three functions: **initialises** class variables, by assigning their values by input commands or by default commands, and adding some self variables that are simply pawns in helping the whole funcitonality of the class; **fitting** model, by taking an X and Y dataset, combining, training base estimators on cross validated datasets of these and outputting predictions which are to be used in training the stack estimator on the resulting so-called Z dataset. The base estimators are then trained on bootstrapped samples of the whole training set. Finally **predicting** the response values of a test set by using predictions by base estimators and a stack model to predict the responses using these predictions. Note I constructed the predict function in similar form to the fit function, but frankly, there is absolutely no need for folds in the predict function, as there are no responses to fit to, so it would be more efficient to simply predict the entire test set.

In [ ]:
class SuperLearnerClassifier(BaseEstimator, ClassifierMixin):  
    """
    SuperLearnerClassifier Class
    """
    
    def __init__(self, base_model_list = ['dt','dt','dt'], \
                 stack_model = 'dt', type_output_labels = 'predictions', \
                 add_orgdata = 'yes', bootstrap_size = 0.8, rand_seed = 3):
        self.base_model_list=base_model_list
        self.stack_model=stack_model
        self.type_output_labels=type_output_labels
        self.add_orgdata=add_orgdata
        self.bootstrap_size = bootstrap_size
        self.rand_seed=rand_seed
        self.num_base_estimators = len(self.base_model_list)
        # Define stack model and base models (up to 10) to be used later
        self.Zdata = None
        self.ult_model = None
        self.model_list = []
        for i in range(1,11):
            self.model_list.append('model'+str(i))
        for model in self.model_list:
            setattr(self, model, None)

    def fit(self,X_train_data,Y_train_data):        
        X_train_data = pd.DataFrame(data=X_train_data)
        Y_train_data = pd.DataFrame(data=Y_train_data)
        X_train_data = X_train_data.reset_index(drop=True)
        Y_train_data = Y_train_data.reset_index(drop=True)
        train_data = pd.merge(Y_train_data, X_train_data, left_index=True, right_index=True)
        # so reset X and Y and merge to one so response values
        # are in correct order so easy to train
        folds_dict = Divide_Into_Folds(train_data,self.num_base_estimators,self.rand_seed)
        # returns dict of folds, no of folds determined by no. of base estimators
        for k in range(self.num_base_estimators):
            other_keys = [m for m in folds_dict.keys() if m!=k+1]
            appended_list = [folds_dict[m] for m in other_keys[1:]]
            train_set = folds_dict[other_keys[0]].append(appended_list,ignore_index=True)
            test_set = folds_dict[k+1]
            # train set is amalgamated all folds other than k
            # test set is kth fold
            for n,each_model in enumerate(self.base_model_list):
                model = BaseModelName(each_model)
                # initialise new model according to string in list
                model = model.fit(train_set[train_set.columns[1:]],train_set[train_set.columns[0]])
                if self.type_output_labels == 'probabilities':
                    y_pred = model.predict_proba(test_set[test_set.columns[1:]])
                else:
                    y_pred = model.predict(test_set[test_set.columns[1:]])
                # if output labels are probabilities use predict_proba otherwise predict
                y_pred = pd.DataFrame(data=y_pred)
                if n+1 == 1:
                    temp_pred = y_pred
                else:
                    temp_pred = pd.merge(temp_pred, y_pred, left_index=True, right_index=True)
                # for each fold, merge predictions from each base estimator
                # if 1st output labels then initialise, otherwise add to existing list
            if k+1 == 1:
                all_pred = temp_pred
            else:
                all_pred = all_pred.append(temp_pred,ignore_index=True)
            # for each fold, merge predictions from last fold, to get set of predictions
            # for all rows
        full_data = folds_dict[1].append([folds_dict[m] for m in range(2,self.num_base_estimators+1)],ignore_index=False)
        # combine folds to new dataset that is in order of predictions above
        if self.add_orgdata == 'no':
            responses = pd.DataFrame(full_data[full_data.columns[0]])
            new_train = pd.merge(responses, all_pred, left_index=True, right_index=True)
        else:
            new_train = pd.merge(full_data, all_pred, left_index=True, right_index=True)
        # if not using original data in dataset Z to accompany output labels of base estimators
        # then merge with X data to get the right order then exclude those cols
        # if use original data too then merge with X data
        new_train.sort_index(inplace=True)
        new_train.reset_index(drop=True)
        # sort training set so outputted predictions are in same order as input data
        self.ult_model = BaseModelName(self.stack_model)
        self.ult_model = self.ult_model.fit(new_train[new_train.columns[1:]],new_train[new_train.columns[0]])
        # initialise and set stack model
        for m,each_model in enumerate(self.base_model_list):
            setattr(self, 'model'+str(m+1), BaseModelName(each_model))
            bootstrap_sample = Bootstrap(train_data,self.bootstrap_size)
            setattr(self, 'model'+str(m+1), getattr(self, 'model'+str(m+1)).\
                        fit(bootstrap_sample[bootstrap_sample.columns[1:]],bootstrap_sample[bootstrap_sample.columns[0]]))
        # train each of the base estimators on bootstrapped samples of original data
        # to be used in predicting test set
        return(self)

    def predict(self,X_test_data):
        # organised in very similar fashion to fit function so comments there
        # hopefully satisfice to explain the thoughts behind the code
        X_test_data = pd.DataFrame(data=X_test_data)
        X_test_data = X_test_data.reset_index(drop=True)
        folds_dict = Divide_Into_Folds(X_test_data,self.num_base_estimators,self.rand_seed)
        for k in range(self.num_base_estimators):
            test_set = folds_dict[k+1]
            for n,model in enumerate(self.base_model_list):
                if self.type_output_labels == 'probabilities':
                    y_pred = getattr(self, 'model'+str(n+1)).predict_proba(test_set)
                else:
                    y_pred = getattr(self, 'model'+str(n+1)).predict(test_set)
                y_pred = pd.DataFrame(data=y_pred)
                if n+1 == 1:
                    temp_pred = y_pred
                else:
                    temp_pred = pd.merge(temp_pred, y_pred, left_index=True, right_index=True)                
            if k+1 == 1:
                all_pred = temp_pred
            else:
                all_pred = all_pred.append(temp_pred,ignore_index=True)
        full_data = folds_dict[1].append([folds_dict[m] for m in range(2,self.num_base_estimators+1)],ignore_index=False)
        self.Zdata = pd.merge(full_data,all_pred, how='inner', left_index=True,right_index=True )
        self.Zdata.sort_index(inplace=True)
        self.Zdata.reset_index(drop=True)
        if self.add_orgdata == 'no':
            col = full_data.shape[1]
            self.Zdata = self.Zdata[self.Zdata.columns[col:]]
        predictions = self.ult_model.predict(self.Zdata)
        return(predictions)

### Q1.
Well above contains all the functionality for many tasks, including by altering class/function inputs one can change the base estimators, the stack estimator, the type of output labels, and whether to include the original data values in the Z dataset.
For example, below contains the SuperLearner with 6 fixed base estimators - decision tree, logistic regression, naive bayes, k nearest neighbours, multi-layer perceptron, support vector machines respectively (denoted by the shorted strings). The hyperparameters are set in the BaseModelName function and can be altered through that function. I did not play around too much with these default parameters though. The stack estimator below is 'dt', a decision tree.

In [ ]:
sl = SuperLearnerClassifier(base_model_list = ['dt','lr','nb'], \
                            stack_model = 'dt',type_output_labels = 'predictions',\
                            add_orgdata = 'no', bootstrap_size = 0.8, rand_seed = 30)

### Q2.

In [ ]:
scores = cross_val_score(sl, X_train, Y_train, cv=3)

Because there is 10 classes it would not be extremely insightful to examine the TPR and FPR, or ROC or DET as there would be 10 different situations to consider. However, a simple measure of performance, that is a good evaluator is the total classification accuracy. The average classification accuracy, and accuracy achieved from each run is displayed below.

In [ ]:
print(sum(scores)/len(scores))

This result is particularly bad. While there may not be useful measures of accuracy immediately avaliable, one can always compare with other estimators. For instance, ZeroR - picking the most prominent class every occasion would simply yield a CA of 10% as our sample is stratified. Random Sampling would also yield roughly 10%. The SuperLearner is only slightly better than these which is worrying. Naive Bayes does better, as shown below. a simple decision tree performs much better, 20NN even better than that, and MLP a further upgrade. And all those were only trained on 66.7% of dataset compared with 90% before.

In [ ]:
dec_tree = BaseModelName('dt')
scores_dt = cross_val_score(dec_tree, X_train, Y_train, cv=3)
print(sum(scores_dt)/len(scores_dt))  
nav_bayes = BaseModelName('nb')
scores_nb = cross_val_score(nav_bayes, X_train, Y_train, cv=3)
print(sum(scores_nb)/len(scores_nb))  
knn = BaseModelName('knn')
scores_knn = cross_val_score(knn, X_train, Y_train, cv=3)
print(sum(scores_knn)/len(scores_knn)) 
mlp = BaseModelName('mlp')
scores_mlp = cross_val_score(mlp, X_train, Y_train, cv=3)
print(sum(scores_mlp)/len(scores_mlp))

Simple check on the program:

In [ ]:
sl1 = SuperLearnerClassifier(base_model_list = ['dt','knn','mlp'], \
                            stack_model = 'dt',type_output_labels = 'predictions',\
                            add_orgdata = 'no', bootstrap_size = 0.8, rand_seed = 30)
sl1.fit(X_train,Y_train)
pred_sl = sl1.predict(X_test)
print(metrics.accuracy_score(Y_test, pred_sl))
pred_dt_sl = sl1.model1.predict(X_test)
print(metrics.accuracy_score(Y_test, pred_dt_sl))
pred_knn_sl = sl1.model2.predict(X_test)
print(metrics.accuracy_score(Y_test, pred_knn_sl))
pred_mlp_sl = sl1.model3.predict(X_test)
print(metrics.accuracy_score(Y_test, pred_mlp_sl))
pred_dt_sl = pd.DataFrame(pred_dt_sl)
pred_knn_sl = pd.DataFrame(pred_knn_sl)
pred_mlp_sl = pd.DataFrame(pred_mlp_sl)
pred_sl_combined = pd.merge(pred_dt_sl,pred_knn_sl, how='inner', left_index=True,right_index=True )
pred_sl_combined = pd.merge(pred_sl_combined,pred_mlp_sl, how='inner', left_index=True,right_index=True )
final_predictions = sl1.ult_model.predict(pred_sl_combined)
print(metrics.accuracy_score(Y_test, final_predictions))

Perhaps it simply is not as predictive as first thought. Each of the individually trained models work well, but in conjunction, and without the original data, the prediction is not robust at all.

### Q3. & Q4.

So as mentioned before the original class incorporates all the functionality. The variable type_output_labels controls whether the output labels from the base estimators are probability distributions for the classes or predictions. The possible inputs are 'predictions' or 'probabilities', with the default being 'predictions'. The variable stack_model takes as its input one of 'dt','lr','nb','svm','mlp', or 'knn', denoting decision tree, logistic regression, naive bayes, support vector machines, multi-layer perceptron, k-nearest neighbours respectively. The hyperparameters could be implemented by altering the BaseModelName input in the class, but this was not necessary. Also, in hindsight, it would have been more efficient to have a true or false parameter rather than a string for the type_output_labels.

In [ ]:
# This helps to display the variable inputs
SuperLearnerClassifier(base_model_list = ['dt','lr','nb','knn','mlp','svm'], \
                            stack_model = 'dt',type_output_labels = 'probabilities',\
                            add_orgdata = 'no', bootstrap_size = 0.8, rand_seed = 30)

### Q5. 
I implemented 4 models, all having stack models trained only on the outputs from the base estimators - none of the original data. Also, all models had the same base estimators - decision tree, logistic regression, 20NN, multilayer perceptron. I thought it fair to include a decision tree and a logistic regression model in the base estimators as this may or may not be compatible with the stack model when the stack model is of the same form. However, I am unsure whether this relationship holds. Nonetheless, it has been controlled for, and as a result will not effect the experiment. The 4 models are simply the 4 combinations possible between the two stack models (decision trees and logistic regression) and the type of output labels (namely predictions and probabilities). Perhaps it would have been more robust a coding exercise and analysis if I simply conducted a grid search with only these 4 models as iteratives so that is done after the analysis below.

In [ ]:
pred_dt = SuperLearnerClassifier(base_model_list = ['dt','lr','knn','mlp'], \
                            stack_model = 'dt',type_output_labels = 'predictions',\
                            add_orgdata = 'no', bootstrap_size = 0.8, rand_seed = 30)
pred_lr = SuperLearnerClassifier(base_model_list = ['dt','lr','knn','mlp'], \
                            stack_model = 'lr',type_output_labels = 'predictions',\
                            add_orgdata = 'no', bootstrap_size = 0.8, rand_seed = 30)
prob_dt = SuperLearnerClassifier(base_model_list = ['dt','lr','knn','mlp'], \
                            stack_model = 'dt',type_output_labels = 'probabilities',\
                            add_orgdata = 'no', bootstrap_size = 0.8, rand_seed = 30)
prob_lr = SuperLearnerClassifier(base_model_list = ['dt','lr','knn','mlp'], \
                            stack_model = 'lr',type_output_labels = 'probabilities',\
                            add_orgdata = 'no', bootstrap_size = 0.8, rand_seed = 30)
pred_dt = pred_dt.fit(X_train, Y_train)
pred_lr = pred_lr.fit(X_train, Y_train)
prob_dt = prob_dt.fit(X_train, Y_train)
prob_lr = prob_lr.fit(X_train, Y_train)
yhat_pred_dt = pred_dt.predict(X_test)
yhat_pred_lr = pred_lr.predict(X_test)
yhat_prob_dt = prob_dt.predict(X_test)
yhat_prob_lr = prob_lr.predict(X_test)
print(metrics.accuracy_score(Y_test, yhat_pred_dt))
print(metrics.accuracy_score(Y_test, yhat_pred_lr))
print(metrics.accuracy_score(Y_test, yhat_prob_dt))
print(metrics.accuracy_score(Y_test, yhat_prob_lr))
pd.crosstab(np.array(Y_test), yhat_pred_dt, rownames=['True'], colnames=['Predicted'], margins=True, dropna = False)
pd.crosstab(np.array(Y_test), yhat_pred_lr, rownames=['True'], colnames=['Predicted'], margins=True, dropna = False)
pd.crosstab(np.array(Y_test), yhat_prob_dt, rownames=['True'], colnames=['Predicted'], margins=True, dropna = False)
pd.crosstab(np.array(Y_test), yhat_prob_lr, rownames=['True'], colnames=['Predicted'], margins=True, dropna = False)

So it seems that output labels as predictions are more conducive to accurate predictions when the stack model is the decision tree (total classification accuracy rises from 9.64% to 11.29% when adopting predictions rather than probabilities), and the opposite is true for logistic regression (classification drops from 10.5% to 9.58%). This muddles the picture, however, in trying to explain this phenomenon, one could remark that logistic regression models can easily implement greater numbers of variables (that have predictive value) into its formulation. In contrast, decision trees produce much better results when they use only smaller sets of variables (that have greater predictive ability). Also, in either choice of output labels, decision trees perform better than the logistic regression models. Although, each of the models do barely better than random guessing. Also, note how many '6's were predicted (285) versus '8's (20).  

### Q6.
So the SuperLearnerClassifier takes as its input a list of strings denoting which base models to use. An example of this is displayed below. There are 6 possible base estimators; their string names are 'dt','knn','nb','mlp','svm','lr' denoting decision tree, k nearest neighbours, naive bayes, multi-layer perceptron, support vector machines and logistic regression respectively. However, any possible combination of these is possible up to a limit of 10 models, but this could easily be extended. The default is three decision trees. Also, there is no need to provide the number of base estimators as the class can simply count the number of strings inputted. The hyperparameters can be altered by adjusting the input to the BaseModelName function but this alteration was not implemented through the SuperLearner class.

In [ ]:
# This helps to display the variable inputs
SuperLearnerClassifier(base_model_list = ['dt','lr','knn','mlp'], \
                            stack_model = 'dt',type_output_labels = 'predictions',\
                            add_orgdata = 'no', bootstrap_size = 0.8, rand_seed = 30)

### Q7.
The Powerset function uses the set notation to produce all the possible combinations of base estimators (with at least 2 base estimators) that could form the SuperLearner. Heterogeneous combinations are the best so it is perhaps frivolous to consider combinations that include the same base estimator, especially considering the weak performance of my computer.

In [ ]:
def Powerset(iterable):
    """
    Returns powerset by with restraint that no. of elements >= 2
    so Powerset([1,2,3]) --> (1,2) (1,3) (2,3) (1,2,3)
    """
    constituents = list(iterable)
    return chain.from_iterable(combinations(constituents,n) \
                               for n in range(2,len(constituents)+1))

all_possible_base_model_combos = list(Powerset(['dt','lr','knn','mlp','nb'])) # svm omitted due to no predict_proba() function

Note I omitted svm as it did not have the predict_proba() function for some reason.I did not run the grid search through all possible combinations of base estimator alliances as that would take an inordinate length of time to complete (342 combinations of >2 different base estimators when combined with iterating over all the values of stack estimators and output labels). So I randomly sampled from each number of base estimators (so for example 2 sampled combinations of base estimators could be ['dt','knn'] and ['mlp',nb']); 2 combinations of 2 base estimators, 2 of 3, 2 of 4, and 1 of 5. The higher combinations having less samples as there are less of them. The grid search also iterated over stack estimators (7), and outputs to train the stack model on (2). 

In [ ]:
all_combinations = [list(combo) for combo in all_possible_base_model_combos]
models2, models3, models4, models5 = [], [], [], []
for i in range(2,6):
    for combo in all_combinations:
        if len(combo)==i:
            locals()["models"+str(i)] += [combo]

models2 = [models2[j] for j in sorted(random.sample(range(len(models2)), 2))]
models3 = [models3[j] for j in sorted(random.sample(range(len(models3)), 2))]
models4 = [models4[j] for j in sorted(random.sample(range(len(models4)), 2))]
# models5 only contains one element so no need to sample from
        
some_combinations = models2+models3+models4+models5
param_grid ={'base_model_list': [model_list for model_list in some_combinations], \
             'stack_model': ['dt','knn','nb','lr','svm','mlp'], \
             'type_output_labels': ['predictions','probabilities'],\
             'bootstrap_size' : [0.65],
             'add_orgdata': ['no'], 'rand_seed': [3]}
grid_search = GridSearchCV(SuperLearnerClassifier(), param_grid, cv=2, \
                             verbose = 1, return_train_score=True)
grid_search.fit(X_train, Y_train)
print("Best parameters set found on development set:")
display(grid_search.best_params_)
display(grid_search.best_score_)
display(grid_search.cv_results_)


So again poor classification accuracy - in all runs, the best was only 10.88%, slightly better than random chance. The best model list was only 3 models long. It would have been expected for this to return the 5 model list, as the more models in an ensemble the better. However, given that it returned a lesser amount, it must be that the mlp and knn models' prediction patterns were co-linear with the original ensemble members, so no new diversity is added, and the extra models are redundant, so they are removed. Also, the stack model returned was Naive Bayes, and type of output labels, predictions. 

### Q8.


I performed a simple test with the only change between the variables being the addition of the original dataset to the output labels that the stack model is trained on. All other parameters are as they are in Q7.'s best model.

In [ ]:
# Set parameters to best ones of Q7
sl_withorgdata = SuperLearnerClassifier(base_model_list = ['dt','lr','nb'], \
                            stack_model = 'nb',type_output_labels = 'predictions',\
                            add_orgdata = 'yes', bootstrap_size = 0.65, rand_seed = 30)
sl_withorgdata = sl_withorgdata.fit(X_train, Y_train)
y_sl_withorgdata = sl_withorgdata.predict(X_test)
print(metrics.accuracy_score(Y_test, y_sl_withorgdata))
sl_withoutorgdata = SuperLearnerClassifier(base_model_list = ['dt','lr','nb'], \
                            stack_model = 'nb',type_output_labels = 'predictions',\
                            add_orgdata = 'no', bootstrap_size = 0.65, rand_seed = 30)
sl_withoutorgdata = sl_withoutorgdata.fit(X_train, Y_train)
y_sl_withoutorgdata = sl_withoutorgdata.predict(X_test)
print(metrics.accuracy_score(Y_test, y_sl_withoutorgdata))

As expected, since the predictive ability of the output labels were so low, adding the original data helped enormously in the predictions.

### Q9.

Due to the preponderance of classes, 10 in number, the efficacy of such analyses or graphs as ROC, DET, TPR, FPR, F-values, to measure performance, are significantly reduced as one must exami ne each class individually. This also excludes Mc Nemar's test which can evaluate the success rate of prediction or correlation of predictions from two models, especially since the observations are the same, so essentially paired samples. The best method of performance evaluation is probably total classification accuracy. This classification accuracy could be compared with random sampling, ZeroR, and some basic models like a decision tree, or k nearest neighbours, for example. The problem with measuring correlation is that the predictions from the models are categorical so we cannot use Pearson's Correlation Coefficient, for example. Even if we were to compute some measure based on the numeric classes, this would be fallacious as the numbers assigned to the classes are simply arbitrary since they are unordered. This also rules out QQ plots to check if they come from the same distribution. Thus, the best measure would be to measure the amount of matching between predictions, calculated in the same way one would find the classification accuracy - the only difference being that instead of response values, there would the other model's predictions. However, there still are great ways to measure correlation (and performance), for instance the two I have implemented below are Cohen's Kappa which measures inter-agreement between predictions, and Mutual Information which measures the amount of information that can be obtained about one model's predictions by the other. 

Let p(x,y) = the proportion of all prediction combinations between model 1 and model 2 where both have the predictions of class labels x and y simultaneously, where x, y is an element of the set of class labels {0,1,2,3,4,5,6,7,8,9}. Let p(x) be the proportion of predictions by model 1 that are of class label x, and similarly for model 2 for p(y). Then Mutual information between model 1 and model 2 is given by the sum of (p(x,y) multiplied by logbase2(p(x,y)/[p(x)p(y)])) over all possible prediction combinations x, y. If x and y are independent, and so uncorrelated then p(x,y) = p(x)p(y) and so MI = 1. If x and y are completely deterministic, for example, an extreme case would be that x is the exactly same variable as y, then p(x,y) = p(y), and we are summing over p(y) multiplied by logbase2(1/p(y)) = -p(y) multiplied by logbase2(p(y)) which is the entropy of model 2's predictions. This value should be close to 1 as they data is stratified, but this is not always the case, as models tend to favour predicting over class over others sometimes. Nonetheless, MI is a value in the set [0,1], with higher values indicating greater correlation between the predictions. Note instead of the log being to the base 2 in the package metrics from sklearn, it is scaled using a natural log, so the range [0,1] is replaced by [0,log(2)=0.69314]

However, the above method suffers from the when a model tends to predict one class with greater frequency than the others. A more robust method is Cohen's Kappa. It is calculated by k = (Pr(a)-Pr(e))/(1-Pr(e)), where Pr(a) = probability of agreement, so this can be the proportion of predictions the two models had agreed on, and Pr(e) = the probability of a chance agreement, so since there are 10 classes, at any one time, there should be a 10% chance of agreement, even if the models were independent. Thus, we would expect 10% of all predictions to be the same if the two models' predictions were independent. Thus, Cohen's kappa is between 0 when Pr(a)=Pr(e), i.e. only chance agreement, and 1 when Pr(a)=1, i.e. agreement in every prediction. This measure is much preferred over our initial way of calculating correlation between predictions (by measuring a kind of classification accuracy with the responses as just the other model's predictions) because the aforementioned measure does not allow for chance agreement.

Note both of these methods are obviously symmetric, so even though MI has links to KL Divergence (which measures how similar the distribution of predictions are between the two models), MI is does not suffer from KL's problems of fragility and prone to different results based on order of model's predictions (i.e. not symmetric). 

In [ ]:
pred1 = sl_withorgdata.model1.predict(X_test)
pred2 = sl_withorgdata.model2.predict(X_test)
pred3 = sl_withorgdata.model3.predict(X_test)
# predictions from before: y_sl_withorgdata
# response values: Y_test

print("We want high correlation between each model's predictions and the response values but low correlation between each other...\n")
print("Proportion of Matching Predictions (CA): [0,1] (higher => more correlated)")
print(metrics.accuracy_score(pred1,pred2))
print(metrics.accuracy_score(pred1,pred3))
print(metrics.accuracy_score(pred1,y_sl_withorgdata))
print(metrics.accuracy_score(pred1,Y_test))
print(metrics.accuracy_score(pred2,pred3))
print(metrics.accuracy_score(pred2,y_sl_withorgdata))
print(metrics.accuracy_score(pred2,Y_test))
print(metrics.accuracy_score(pred3,y_sl_withorgdata))
print(metrics.accuracy_score(pred3,Y_test))
print(metrics.accuracy_score(y_sl_withorgdata,Y_test))

print("\nMutual Information Scores: [0,0.6913] (higher => more correlated)")
print(metrics.mutual_info_score(pred1,pred2))
print(metrics.mutual_info_score(pred1,pred3))
print(metrics.mutual_info_score(pred1,y_sl_withorgdata))
print(metrics.mutual_info_score(pred1,Y_test))
print(metrics.mutual_info_score(pred2,pred3))
print(metrics.mutual_info_score(pred2,y_sl_withorgdata))
print(metrics.mutual_info_score(pred2,Y_test))
print(metrics.mutual_info_score(pred3,y_sl_withorgdata))
print(metrics.mutual_info_score(pred3,Y_test))
print(metrics.mutual_info_score(y_sl_withorgdata,Y_test))

print("\nCohen's Kappa Scores: [0,1] (higher => more correlated)")
print(metrics.cohen_kappa_score(pred1,pred2))
print(metrics.cohen_kappa_score(pred1,pred3))
print(metrics.cohen_kappa_score(pred1,y_sl_withorgdata))
print(metrics.cohen_kappa_score(pred1,Y_test))
print(metrics.cohen_kappa_score(pred2,pred3))
print(metrics.cohen_kappa_score(pred2,y_sl_withorgdata))
print(metrics.cohen_kappa_score(pred2,Y_test))
print(metrics.cohen_kappa_score(pred3,y_sl_withorgdata))
print(metrics.cohen_kappa_score(pred3,Y_test))
print(metrics.cohen_kappa_score(y_sl_withorgdata,Y_test))

All three measures provide similar rankings. The three base models have a greater correlation with the response values than the stack model. The three models are nearly as correlated with each other as they are with the response values.

Since throughout the tasks, the runs generally performed much worse than expected, I ran a simple experiment to validate my answers, and sure enough the result was dissapointing, yet consistent with aforementioned results.

In [ ]:
X_train_data = pd.DataFrame(data=X_train)Y_train_data = pd.DataFrame(data=Y_train)
X_train_data = X_train_data.reset_index(drop=True)
Y_train_data = Y_train_data.reset_index(drop=True)
train_data = pd.merge(Y_train_data, X_train_data, left_index=True, right_index=True)
folds_dict = Divide_Into_Folds(train_data,3,76)
for k in range(3):
    other_keys = [m for m in folds_dict.keys() if m!=k+1]
    appended_list = [folds_dict[m] for m in other_keys[1:]]
    train_set = folds_dict[other_keys[0]].append(appended_list,ignore_index=True)
    test_set = folds_dict[k+1]
    for n,each_model in enumerate(['dt','lr','nb']):
        model = BaseModelName(each_model)
        model = model.fit(train_set[train_set.columns[1:]],train_set[train_set.columns[0]])
        y_pred = model.predict(test_set[test_set.columns[1:]])
        y_pred = pd.DataFrame(data=y_pred)
        if n+1 == 1:
            temp_pred = y_pred
        else:
            temp_pred = pd.merge(temp_pred, y_pred, left_index=True, right_index=True)
    if k+1 == 1:
        all_pred = temp_pred
    else:
        all_pred = all_pred.append(temp_pred,ignore_index=True)
full_data = folds_dict[1].append([folds_dict[m] for m in range(2,4)],ignore_index=False)
responses = pd.DataFrame(full_data[full_data.columns[0]])
new_train = pd.merge(responses, all_pred, left_index=True, right_index=True)
new_train.sort_index(inplace=True)
new_train.reset_index(drop=True)
ult_model = BaseModelName('dt')
ult_model = ult_model.fit(new_train[new_train.columns[1:]],new_train[new_train.columns[0]])
model1 = BaseModelName('dt')
bootstrap_sample = Bootstrap(train_data,0.65)
model1 = model1.fit(bootstrap_sample[bootstrap_sample.columns[1:]],bootstrap_sample[bootstrap_sample.columns[0]])
model2 = BaseModelName('lr')
bootstrap_sample = Bootstrap(train_data,0.65)
model2 = model2.fit(bootstrap_sample[bootstrap_sample.columns[1:]],bootstrap_sample[bootstrap_sample.columns[0]])
model3 = BaseModelName('nb')
bootstrap_sample = Bootstrap(train_data,0.65)
model3 = model3.fit(bootstrap_sample[bootstrap_sample.columns[1:]],bootstrap_sample[bootstrap_sample.columns[0]])

X_test_data = pd.DataFrame(data=X_test)
X_test_data = X_test_data.reset_index(drop=True)
y_pred1 = model1.predict(X_test_data)
y_pred2 = model2.predict(X_test_data)
y_pred3 = model3.predict(X_test_data)
y_pred1 = pd.DataFrame(data=y_pred1)
y_pred2 = pd.DataFrame(data=y_pred2)
y_pred3 = pd.DataFrame(data=y_pred3)
temp_pred = pd.merge(y_pred1, y_pred2, left_index=True, right_index=True)
temp_pred = pd.merge(temp_pred, y_pred3, left_index=True, right_index=True)
full_pred = ult_model.predict(temp_pred)
print(metrics.accuracy_score(Y_test, full_pred))


Also, I feared the order may be jumbled in the predictions, but that is not the reason, as below the order is maintained after all the operations.

In [ ]:
check = 'No problems'
for i in range(len(new_train)):
    if new_train["label"].loc[i] != train_data["label"].loc[i]:
        check = 'Problems'
    if new_train["0_x"].loc[i] != all_pred["0_x"].loc[i]:
        check = 'Other Problems'
print(check)